<a href="https://colab.research.google.com/github/ssundxr/Neural-network-projects/blob/main/MobilenetV2-bayesian-optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt


In [7]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1"

# Check directory structure
print("Dataset Contents:", os.listdir(dataset_path))


Dataset Contents: ['Animals', 'README.md']


In [9]:
train_dir = os.path.join(dataset_path, "Animals", "train")
val_dir = os.path.join(dataset_path, "Animals", "validation")

print(f"Updated Train Directory: {train_dir}")
print(f"Updated Validation Directory: {val_dir}")


Updated Train Directory: /root/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1/Animals/train
Updated Validation Directory: /root/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1/Animals/validation


In [12]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1"
print("Dataset Path:", dataset_path)
print("Contents:", os.listdir(dataset_path))


Dataset Path: /root/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1
Contents: ['Animals', 'README.md']


In [13]:
train_dir = os.path.join(dataset_path, "train")
val_dir = os.path.join(dataset_path, "validation")


In [14]:
train_dir = os.path.join(dataset_path, "Animals", "train")
val_dir = os.path.join(dataset_path, "Animals", "validation")


In [15]:
print("Train Directory Exists:", os.path.exists(train_dir))
print("Validation Directory Exists:", os.path.exists(val_dir))


Train Directory Exists: False
Validation Directory Exists: False


In [16]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1"

# Walk through all subdirectories
for root, dirs, files in os.walk(dataset_path):
    print(f"📂 {root}")
    for d in dirs:
        print(f"  📁 {d}")
    for f in files:
        print(f"  📄 {f}")


📂 /root/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1
  📁 Animals
  📄 README.md
📂 /root/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1/Animals
  📁 snakes
  📁 dogs
  📁 cats
📂 /root/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1/Animals/snakes
  📄 2_0313.jpg
  📄 2_0424.jpg
  📄 2_0850.jpg
  📄 2_0095.jpg
  📄 2_0413.jpg
  📄 2_0033.jpg
  📄 2_0330.jpg
  📄 2_0327.jpg
  📄 2_0945.jpg
  📄 2_0010.jpg
  📄 2_0393.jpg
  📄 2_0249.jpg
  📄 2_0397.jpg
  📄 2_0043.jpg
  📄 2_0053.jpg
  📄 2_0788.jpg
  📄 2_0434.jpg
  📄 2_0352.jpg
  📄 2_0643.jpg
  📄 2_0419.jpg
  📄 2_0896.jpg
  📄 2_0609.jpg
  📄 2_0725.jpg
  📄 2_0607.jpg
  📄 2_0965.jpg
  📄 2_0080.jpg
  📄 2_0679.jpg
  📄 2_0658.jpg
  📄 2_0260.jpg
  📄 2_0287.jpg
  📄 2_0989.jpg
  📄 2_0143.jpg
  📄 2_0504.jpg
  📄 2_0520.jpg
  📄 2_0517.jpg
  📄 2_0160.jpg
  📄 2_0648.jpg
  📄 2_0705.jpg
  📄 2_0262.jpg
  📄 2_0823.jpg
  📄 2_0600.jpg
  📄 2_0044.jpg
  📄 2_0415

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

dataset_path = "/root/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1/Animals"

# Data generators with automatic train-validation split
datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

# Training data (80%)
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training"  # Training subset
)

# Validation data (20%)
val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"  # Validation subset
)


Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [18]:
!pip install tensorflow keras-tuner


In [19]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from kerastuner.tuners import BayesianOptimization

# Define the model function
def build_mobilenetv2(hp):
    base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze base model layers

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(hp.Int('units', min_value=64, max_value=256, step=64), activation='relu')(x)
    x = Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1))(x)
    output = Dense(3, activation='softmax')  # 3 classes (snakes, dogs, cats)

    model = Model(inputs=base_model.input, outputs=output)

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


<ipython-input-19-abbd3c2bc097>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import BayesianOptimization


In [22]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from kerastuner.tuners import BayesianOptimization

# Define the model function
def build_mobilenetv2(hp):
    base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze base model layers

    x = base_model.output  # Get the output of the base model
    x = GlobalAveragePooling2D()(x)  # Pooling to reduce dimensions
    x = Dense(hp.Int('units', min_value=64, max_value=256, step=64), activation='relu')(x)
    x = Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1))(x)
    output = Dense(3, activation='softmax')(x)  # 3 classes (snakes, dogs, cats)

    model = Model(inputs=base_model.input, outputs=output)  # Ensure input-output connection

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [24]:
tuner = BayesianOptimization(
    build_mobilenetv2,
    objective='val_accuracy',
    max_trials=1,  # Number of different models to try
    executions_per_trial=1,
    directory='bayesian_opt_mobilenetv2',
    project_name='animal_classification'
)

tuner.search(train_data, validation_data=val_data, epochs=1)


Trial 1 Complete [00h 02m 32s]
val_accuracy: 0.9783333539962769

Best val_accuracy So Far: 0.9783333539962769
Total elapsed time: 00h 02m 32s


In [26]:
tuner.search(
    train_data,
    validation_data=val_data,
    epochs=10,  # You can increase this later
    verbose=1
)


In [27]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best number of units in dense layer: {best_hps.get('units')}")
print(f"Best dropout rate: {best_hps.get('dropout')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")


Best number of units in dense layer: 192
Best dropout rate: 0.30000000000000004
Best learning rate: 0.001722921830384839


In [28]:
best_model = tuner.hypermodel.build(best_hps)

history = best_model.fit(
    train_data,
    validation_data=val_data,
    epochs=2,  # Adjust based on performance
    verbose=1
)


Epoch 1/2
75/75 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.8688 - loss: 0.3474 - val_accuracy: 0.9783 - val_loss: 0.0868
Epoch 2/2
75/75 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.9852 - loss: 0.0464 - val_accuracy: 0.9800 - val_loss: 0.0766


In [29]:
val_loss, val_acc = best_model.evaluate(val_data)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


19/19 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.9845 - loss: 0.0594
Validation Accuracy: 98.00%


In [31]:
best_model.save("best_mobilenetv2_model.keras")
